# Running a QiskitPattern remotely

In this tutorial, we will write a basic QiskitPattern using Quantum Serverless. We will show how to run the pattern remotely and retrieve the results from the serverless client.

### Writing the QiskitPattern

First, we need to write the pattern code and save it to a file called [pattern.py](./source_files/pattern.py). This pattern creates a two-qubit quantum circuit that prepares a Bell state, measures the result, and saves the measured probability distribution.

The code for the pattern is shown below:

```python
# source_files/pattern.py

from qiskit import QuantumCircuit
from qiskit.primitives import Sampler

from quantum_serverless import save_result

# all print statement will be available in job logs
print("Running pattern...")

# creating circuit
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.measure_all()

# running Sampler primitive
sampler = Sampler()
quasi_dists = sampler.run(circuit).result().quasi_dists

# save results of pattern execution, 
# which will be accessible by calling `.result()`
save_result(quasi_dists)
print("Completed running pattern.")
```

### Deploying the QiskitPattern

To run the pattern, we need to import the necessary classes and configure them. One of these classes is QuantumServerless, which is a client class for interacting with compute resources.

QuantumServerless takes a [BaseProvider](https://qiskit-extensions.github.io/quantum-serverless/stubs/quantum_serverless.core.BaseProvider.html) instance as a constructor argument. The provider stores configuration information about our compute resources, such as where they are located and how to connect to them. In this example, we will use a provider that is connected to a local Docker Compose setup. In this case, it allows us to run the pattern locally on our machine. If you want to run the pattern elsewhere, you will need to provide the corresponding host and authentication details.

In [1]:
from quantum_serverless import ServerlessProvider
import os

> &#x26A0; This provider is set up with default credentials to a test cluster intended to run on your machine. For information on setting up infrastructure on your local machine, check out the guide on [local infrastructure setup](https://qiskit-extensions.github.io/quantum-serverless/deployment/local.html).

In [2]:
serverless = ServerlessProvider(
    token=os.environ.get("GATEWAY_TOKEN", "awesome_token"),
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8010"),
)
serverless

<ServerlessProvider: gateway-provider>



`QiskitPattern` accepts couple of required parameters:

- title - name of the program
- entrypoint - name of python file you want to execute
- working_dir - directory where  your script is located (directory size must be less than 50MB). This is optional parameter and will be current folder by default.

> Warning! All content of `working_dir` will be shipped to cluster for execution

In [3]:
from quantum_serverless import QiskitPattern

pattern = QiskitPattern(
    title="my-first-pattern", entrypoint="pattern.py", working_dir="./source_files/"
)

serverless.upload(pattern)

'my-first-pattern'

### Running the QiskitPattern

After deploying the QiskitPattern, we can run the pattern by calling the `run()` method of the [ServerlessProvider](https://qiskit-extensions.github.io/quantum-serverless/stubs/quantum_serverless.core.ServerlessProvider.html#quantum_serverless.core.ServerlessProvider) object:

In [4]:
job = serverless.run("my-first-pattern")
job

<Job | b854e116-65ee-4b7b-acdd-2ca18469b50e>

[Job](https://qiskit-extensions.github.io/quantum-serverless/stubs/quantum_serverless.core.Job.html#quantum_serverless.core.Job) instances have a `status()` method to check status of pattern execution.

In [5]:
job.status()

'QUEUED'

`Job` instances also have a `result()` method for retrieving results. The `result()` method will not return until the job is done running the pattern.

In [6]:
job.result()

[{'0': 0.4999999999999999, '3': 0.4999999999999999}]

To inspect the logs from a pattern, access them from the ``Job`` instance.

In [7]:
print(job.logs())

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
Running pattern...
Completed running pattern.



`ServerlessProvider` object has method `.widget` which renders Jupyter widget to see list of executed programs.

![widget](./images/widget.png)

In [ ]:
serverless.widget()